In [ ]:
# =============================================================================
# ANÁLISIS DE SERIES DE TIEMPO Y PREDICCIÓN DE DENGUE CON PEEPHOLE LSTM
# =============================================================================

# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# PyTorch y LSTM
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer

# Optuna para optimización de hiperparámetros
import optuna
from optuna.trial import TrialState

# Configuración
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")

In [ ]:
# =============================================================================
# FUNCIONES AUXILIARES PARA MANEJO DE NaN
# =============================================================================

def clean_and_impute_data(df, strategy='median'):
    """
    Limpia y maneja valores NaN en el DataFrame
    """
    #print(f"Limpiando datos - Forma inicial: {df.shape}")
    
    # Verificar valores NaN
    nan_counts = df.isnull().sum()
    total_nans = nan_counts.sum()
    
    if total_nans > 0:
        print(f"Valores NaN encontrados: {total_nans}")
        print("Por columna:")
        for col, count in nan_counts[nan_counts > 0].items():
            print(f"  {col}: {count} ({count/len(df)*100:.2f}%)")
        
        # Separar columnas numéricas y categóricas
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        categorical_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()
        
        # Imputar columnas numéricas
        if len(numeric_cols) > 0:
            imputer_numeric = SimpleImputer(strategy=strategy)
            df[numeric_cols] = imputer_numeric.fit_transform(df[numeric_cols])
        
        # Imputar columnas categóricas
        if len(categorical_cols) > 0:
            imputer_categorical = SimpleImputer(strategy='most_frequent')
            df[categorical_cols] = imputer_categorical.fit_transform(df[categorical_cols])
        
        print(f"✓ Valores NaN imputados usando estrategia '{strategy}'")
    
    # Verificar valores infinitos
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    inf_counts = {}
    for col in numeric_cols:
        inf_count = np.isinf(df[col]).sum()
        if inf_count > 0:
            inf_counts[col] = inf_count
            # Reemplazar infinitos con NaN y luego imputar
            df[col] = df[col].replace([np.inf, -np.inf], np.nan)
            df[col] = df[col].fillna(df[col].median())
    
    if inf_counts:
        print(f"Valores infinitos encontrados y corregidos: {inf_counts}")
    
    # Verificación final
    final_nans = df.isnull().sum().sum()
    final_infs = np.isinf(df.select_dtypes(include=[np.number])).sum().sum()
    
    #print(f"✓ Limpieza completada - NaN restantes: {final_nans}, Inf restantes: {final_infs}")
    #print(f"Forma final: {df.shape}")
    
    return df

def validate_tensor_data(tensor, name="tensor"):
    """
    Valida que un tensor no contenga NaN o Inf
    """
    if torch.isnan(tensor).any():
        print(f"¡ADVERTENCIA! {name} contiene valores NaN")
        return False
    if torch.isinf(tensor).any():
        print(f"¡ADVERTENCIA! {name} contiene valores infinitos")
        return False
    return True

In [ ]:
# =============================================================================
# 1. ANÁLISIS EXPLORATORIO DE DATOS - SERIES DE TIEMPO
# =============================================================================

print("="*70)
print("ANÁLISIS EXPLORATORIO DE DATOS - SERIES DE TIEMPO")
print("="*70)

df = pd.read_parquet("../../Datos/df_train.parquet")
# Cargar datos
print("Cargando datos...")
df_train = df[df['anio'] < 2021]  # Filtrar por años recientes
df_test = df[df['anio'] >= 2021]  # Datos de prueba para 2021 en adelante

#df_train = pd.read_parquet("../../Datos/df_train.parquet")
#df_test = pd.read_parquet("../../Datos/df_test.parquet")
sample_submission = pd.read_csv("../../Datos/sample_submission.csv")

print(f"Datos de entrenamiento: {df_train.shape}")
print(f"Datos de prueba: {df_test.shape}")
print(f"Archivo de submisión: {sample_submission.shape}")

# LIMPIEZA INICIAL DE DATOS
print("\n--- LIMPIEZA INICIAL DE DATOS ---")
df_train = clean_and_impute_data(df_train.copy())
df_test = clean_and_impute_data(df_test.copy())

# Exploración inicial
print("\n--- ESTRUCTURA DE DATOS DE ENTRENAMIENTO ---")
print(df_train.info())
print("\nPrimeras filas:")
print(df_train.head())

print("\n--- ESTADÍSTICAS DESCRIPTIVAS ---")
print(df_train.describe())

# Crear columna de fecha para análisis temporal
def create_date_column(df):
    """Crear columna de fecha a partir de año y semana"""
    df = df.copy()
    # Crear fecha usando el primer día de cada semana
    df['fecha'] = pd.to_datetime(df['anio'].astype(str) + '-W' + df['semana'].astype(str) + '-1', 
                                format='%Y-W%U-%w')
    return df

df_train = create_date_column(df_train)
df_test = create_date_column(df_test)

print(f"\nRango temporal entrenamiento: {df_train['fecha'].min()} a {df_train['fecha'].max()}")
print(f"Rango temporal prueba: {df_test['fecha'].min()} a {df_test['fecha'].max()}")

# Análisis por barrio y tiempo
print("\n--- ANÁLISIS TEMPORAL POR BARRIO ---")
print(f"Número de barrios únicos en train: {df_train['id_bar'].nunique()}")
print(f"Número de barrios únicos en test: {df_test['id_bar'].nunique()}")

# Verificar continuidad temporal
barrios_comunes = set(df_train['id_bar'].unique()) & set(df_test['id_bar'].unique())
print(f"Barrios comunes entre train y test: {len(barrios_comunes)}")

In [ ]:
# =============================================================================
# VISUALIZACIONES EXPLORATORIAS
# =============================================================================

# 1. Serie temporal de dengue agregada
print("\n--- GENERANDO VISUALIZACIONES ---")

# Agregar por fecha
dengue_temporal = df_train.groupby('fecha')['dengue'].agg(['sum', 'mean', 'count']).reset_index()

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Análisis Temporal de Dengue', fontsize=16, fontweight='bold')

# Serie temporal total
axes[0,0].plot(dengue_temporal['fecha'], dengue_temporal['sum'], linewidth=2, color='red')
axes[0,0].set_title('Casos Totales de Dengue por Semana')
axes[0,0].set_xlabel('Fecha')
axes[0,0].set_ylabel('Casos Totales')
axes[0,0].tick_params(axis='x', rotation=45)

# Serie temporal promedio
axes[0,1].plot(dengue_temporal['fecha'], dengue_temporal['mean'], linewidth=2, color='blue')
axes[0,1].set_title('Casos Promedio de Dengue por Semana')
axes[0,1].set_xlabel('Fecha')
axes[0,1].set_ylabel('Casos Promedio')
axes[0,1].tick_params(axis='x', rotation=45)

# Distribución de dengue
axes[1,0].hist(df_train['dengue'], bins=50, alpha=0.7, color='green', edgecolor='black')
axes[1,0].set_title('Distribución de Casos de Dengue')
axes[1,0].set_xlabel('Casos de Dengue')
axes[1,0].set_ylabel('Frecuencia')

# Boxplot por año
df_train.boxplot(column='dengue', by='anio', ax=axes[1,1])
axes[1,1].set_title('Distribución de Dengue por Año')
axes[1,1].set_xlabel('Año')
axes[1,1].set_ylabel('Casos de Dengue')

plt.tight_layout()
plt.show()

# 2. Análisis estacional
print("Analizando patrones estacionales...")

# Agregar mes y trimestre
df_train['mes'] = df_train['fecha'].dt.month
df_train['trimestre'] = df_train['fecha'].dt.quarter

# Patrones estacionales
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Por semana del año
dengue_por_semana = df_train.groupby('semana')['dengue'].mean()
axes[0].plot(dengue_por_semana.index, dengue_por_semana.values, marker='o', linewidth=2)
axes[0].set_title('Patrón Estacional por Semana del Año')
axes[0].set_xlabel('Semana')
axes[0].set_ylabel('Casos Promedio de Dengue')
axes[0].grid(True, alpha=0.3)

# Por mes
dengue_por_mes = df_train.groupby('mes')['dengue'].mean()
axes[1].bar(dengue_por_mes.index, dengue_por_mes.values, color='coral', alpha=0.8)
axes[1].set_title('Patrón Estacional por Mes')
axes[1].set_xlabel('Mes')
axes[1].set_ylabel('Casos Promedio de Dengue')
axes[1].set_xticks(range(1, 13))

# Por trimestre
dengue_por_trimestre = df_train.groupby('trimestre')['dengue'].mean()
axes[2].bar(dengue_por_trimestre.index, dengue_por_trimestre.values, color='lightblue', alpha=0.8)
axes[2].set_title('Patrón Estacional por Trimestre')
axes[2].set_xlabel('Trimestre')
axes[2].set_ylabel('Casos Promedio de Dengue')

plt.tight_layout()
plt.show()

# 3. Análisis de correlaciones con variables climáticas
print("Analizando correlaciones con variables climáticas...")

# Variables climáticas
# vars_clima = ['lluvia_mean', 'lluvia_var', 'lluvia_max', 'lluvia_min',
#               'temperatura_mean', 'temperatura_var', 'temperatura_max', 'temperatura_min']

vars_clima = ['lluvia_mean', #'lluvia_var', 'lluvia_max',
              'temperatura_mean', #'temperatura_var', 'temperatura_max', 'temperatura_min'
              ]

# Matriz de correlación
corr_matrix = df_train[vars_clima + ['dengue']].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.3f', cbar_kws={'label': 'Correlación'})
plt.title('Matriz de Correlación - Variables Climáticas vs Dengue', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# 4. Series temporales por barrio (muestra de algunos barrios)
print("Analizando series temporales por barrio...")

# Seleccionar barrios con más datos
barrios_datos = df_train.groupby('id_bar').size().sort_values(ascending=False)
top_barrios = barrios_datos.head(6).index

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, barrio in enumerate(top_barrios):
    data_barrio = df_train[df_train['id_bar'] == barrio].sort_values('fecha')
    axes[i].plot(data_barrio['fecha'], data_barrio['dengue'], linewidth=2)
    axes[i].set_title(f'Barrio {barrio} (n={len(data_barrio)})')
    axes[i].set_xlabel('Fecha')
    axes[i].set_ylabel('Casos de Dengue')
    axes[i].tick_params(axis='x', rotation=45)
    axes[i].grid(True, alpha=0.3)

plt.suptitle('Series Temporales de Dengue por Barrio', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# 5. Análisis de tendencias y autocorrelación
print("Analizando tendencias y autocorrelación...")

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Preparar serie temporal agregada
serie_dengue = dengue_temporal.set_index('fecha')['sum'].asfreq('W-MON')

# Descomposición estacional
fig, axes = plt.subplots(4, 1, figsize=(15, 12))

# Serie original
axes[0].plot(serie_dengue.index, serie_dengue.values, linewidth=2, color='black')
axes[0].set_title('Serie Original de Dengue', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Casos')

# Descomposición
decomp = seasonal_decompose(serie_dengue.dropna(), model='additive', period=52)

axes[1].plot(decomp.trend.index, decomp.trend.values, linewidth=2, color='blue')
axes[1].set_title('Tendencia', fontsize=12)
axes[1].set_ylabel('Tendencia')

axes[2].plot(decomp.seasonal.index, decomp.seasonal.values, linewidth=2, color='green')
axes[2].set_title('Estacionalidad', fontsize=12)
axes[2].set_ylabel('Estacional')

axes[3].plot(decomp.resid.index, decomp.resid.values, linewidth=2, color='red')
axes[3].set_title('Residuos', fontsize=12)
axes[3].set_ylabel('Residuos')
axes[3].set_xlabel('Fecha')

plt.tight_layout()
plt.show()

# Autocorrelación
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

plot_acf(serie_dengue.dropna(), lags=40, ax=axes[0], title='Autocorrelación (ACF)')
plot_pacf(serie_dengue.dropna(), lags=40, ax=axes[1], title='Autocorrelación Parcial (PACF)')

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("RESUMEN DEL ANÁLISIS EXPLORATORIO")
print("="*70)
print(f"✓ Datos de entrenamiento: {df_train.shape[0]:,} observaciones")
print(f"✓ Período de entrenamiento: {df_train['anio'].min()}-{df_train['anio'].max()}")
print(f"✓ Número de barrios: {df_train['id_bar'].nunique()}")
print(f"✓ Variables climáticas disponibles: {len(vars_clima)}")
print(f"✓ Correlación más alta con dengue: {corr_matrix['dengue'].abs().sort_values(ascending=False).index[1]} "
      f"({corr_matrix['dengue'].abs().sort_values(ascending=False).iloc[1]:.3f})")

In [ ]:
# =============================================================================
# 2. IMPLEMENTACIÓN DE PEEPHOLE LSTM CON PYTORCH
# =============================================================================

print("\n" + "="*70)
print("IMPLEMENTACIÓN DE PEEPHOLE LSTM CON PYTORCH")
print("="*70)

# Configurar dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositivo utilizado: {device}")

class TimeSeriesDataset(Dataset):
    """Dataset personalizado para series de tiempo con validación de NaN"""
    
    def __init__(self, data, target_col, feature_cols, seq_length, target_length=1):
        self.data = data.copy()
        self.target_col = target_col
        self.feature_cols = feature_cols
        self.seq_length = seq_length
        self.target_length = target_length
        
        # Preparar datos por barrio
        self.sequences = []
        self.targets = []
        
        #print(f"Creando dataset con {len(feature_cols)} características y secuencias de longitud {seq_length}")
        
        valid_sequences = 0
        invalid_sequences = 0
        
        for id_bar in data['id_bar'].unique():
            barrio_data = data[data['id_bar'] == id_bar].sort_values('fecha')
            
            if len(barrio_data) >= seq_length + target_length:
                features = barrio_data[feature_cols].values
                targets = barrio_data[target_col].values
                
                # Verificar que no haya NaN en las características
                if np.isnan(features).any():
                    print(f"Advertencia: NaN encontrado en características del barrio {id_bar}")
                    continue
                
                if np.isnan(targets).any():
                    print(f"Advertencia: NaN encontrado en targets del barrio {id_bar}")
                    continue
                
                for i in range(len(barrio_data) - seq_length - target_length + 1):
                    seq_x = features[i:i+seq_length]
                    seq_y = targets[i+seq_length:i+seq_length+target_length]
                    
                    # Validación adicional de NaN
                    if not (np.isnan(seq_x).any() or np.isnan(seq_y).any() or 
                           np.isinf(seq_x).any() or np.isinf(seq_y).any()):
                        self.sequences.append(seq_x)
                        self.targets.append(seq_y)
                        valid_sequences += 1
                    else:
                        invalid_sequences += 1
        
        #print(f"✓ Secuencias válidas: {valid_sequences}")
        if invalid_sequences > 0:
            print(f"⚠ Secuencias inválidas descartadas: {invalid_sequences}")
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        seq = torch.FloatTensor(self.sequences[idx])
        target = torch.FloatTensor(self.targets[idx])
        
        # Validación final de tensores
        if not (validate_tensor_data(seq, f"sequence_{idx}") and 
                validate_tensor_data(target, f"target_{idx}")):
            # Reemplazar con tensor de ceros si hay problemas
            seq = torch.zeros_like(seq)
            target = torch.zeros_like(target)
        
        return seq, target

class PeepholeLSTM(nn.Module):
    """Implementación de Peephole LSTM con validación de NaN"""
    
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.1):
        super(PeepholeLSTM, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # Capas LSTM con peephole connections
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                           batch_first=True, dropout=dropout if num_layers > 1 else 0)
        
        # Capas fully connected
        self.fc_layers = nn.ModuleList()
        
        # Primera capa
        self.fc_layers.append(nn.Linear(hidden_size, hidden_size // 2))
        self.fc_layers.append(nn.ReLU())
        self.fc_layers.append(nn.Dropout(dropout))
        
        # Segunda capa
        self.fc_layers.append(nn.Linear(hidden_size // 2, hidden_size // 4))
        self.fc_layers.append(nn.ReLU())
        self.fc_layers.append(nn.Dropout(dropout))
        
        # Capa de salida
        self.fc_layers.append(nn.Linear(hidden_size // 4, output_size))
        
        # Normalización por lotes
        self.batch_norm = nn.BatchNorm1d(hidden_size)
        
    def forward(self, x):
        # Validar entrada
        if not validate_tensor_data(x, "input"):
            print("Entrada contiene valores inválidos, reemplazando con ceros")
            x = torch.zeros_like(x)
        
        # Inicializar estados ocultos
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # LSTM forward
        lstm_out, _ = self.lstm(x, (h0, c0))
        
        # Tomar la última salida
        lstm_out = lstm_out[:, -1, :]
        
        # Validar salida LSTM
        if not validate_tensor_data(lstm_out, "lstm_output"):
            print("Salida LSTM contiene valores inválidos")
            lstm_out = torch.zeros_like(lstm_out)
        
        # Normalización por lotes
        if lstm_out.size(0) > 1:  # BatchNorm requiere más de 1 muestra
            lstm_out = self.batch_norm(lstm_out)
        
        # Capas fully connected
        out = lstm_out
        for i, layer in enumerate(self.fc_layers):
            out = layer(out)
            # Validar después de cada capa
            if not validate_tensor_data(out, f"fc_layer_{i}"):
                print(f"Salida de capa FC {i} contiene valores inválidos")
                out = torch.clamp(out, -1e6, 1e6)  # Clamp valores extremos
        
        return out

def prepare_data_for_model(df_train, df_test, feature_cols, target_col, seq_length):
    """Preparar datos para el modelo con manejo robusto de NaN"""
    
    #print("Preparando datos para el modelo...")
    
    # Limpiar datos antes de procesar
    # Filtrar datos de entrenamiento y prueba por año
    df_train_clean = clean_and_impute_data(df_train[df_train['anio'] < 2021].copy())
    df_test_clean = clean_and_impute_data(df_train[df_train['anio'] >= 2021].copy())
    
    # Combinar datos para normalización consistente
    combined_data = pd.concat([df_train_clean, df_test_clean], ignore_index=True)
    
    # Verificar que las columnas existan
    missing_features = [col for col in feature_cols if col not in combined_data.columns]
    if missing_features:
        print(f"⚠ Características faltantes: {missing_features}")
        feature_cols = [col for col in feature_cols if col in combined_data.columns]
        print(f"Usando características disponibles: {feature_cols}")
    
    # Normalización de características con manejo de NaN
    #print("Normalizando características...")
    scaler_features = StandardScaler()
    scaler_target = StandardScaler()
    
    # Verificar datos antes de normalizar
    #print(f"Datos antes de normalizar - NaN en features: {combined_data[feature_cols].isnull().sum().sum()}")
    #print(f"Datos antes de normalizar - NaN en target: {combined_data[target_col].isnull().sum()}")
    
    # Ajustar escaladores con datos combinados
    try:
        combined_data[feature_cols] = scaler_features.fit_transform(combined_data[feature_cols])
        combined_data[[target_col]] = scaler_target.fit_transform(combined_data[[target_col]])
    except Exception as e:
        print(f"Error durante normalización: {e}")
        # Limpiar nuevamente si hay problemas
        combined_data = clean_and_impute_data(combined_data)
        combined_data[feature_cols] = scaler_features.fit_transform(combined_data[feature_cols])
        combined_data[[target_col]] = scaler_target.fit_transform(combined_data[[target_col]])
    
    # Verificar después de normalizar
    #print(f"Datos después de normalizar - NaN en features: {combined_data[feature_cols].isnull().sum().sum()}")
    #print(f"Datos después de normalizar - NaN en target: {combined_data[target_col].isnull().sum()}")
    
    # Separar nuevamente
    train_scaled = combined_data.iloc[:len(df_train_clean)].copy()
    test_scaled = combined_data.iloc[len(df_train_clean):].copy()
    
    # Crear datasets
    train_dataset = TimeSeriesDataset(train_scaled, target_col, feature_cols, seq_length)
    test_dataset = TimeSeriesDataset(test_scaled, target_col, feature_cols, seq_length)
    
    #print(f"Secuencias de entrenamiento: {len(train_dataset)}")
    #print(f"Secuencias de prueba: {len(test_dataset)}")
    
    return train_dataset, test_dataset, scaler_features, scaler_target

# Definir características para el modelo
# feature_cols = ['ESTRATO', 'area_barrio', 'concentraciones', 'vivienda', 
#                 'equipesado', 'sumideros', 'maquina'] + vars_clima

feature_cols = ['area_barrio', 'concentraciones',  
                'equipesado'] + vars_clima

target_col = 'dengue'

print(f"Características seleccionadas: {len(feature_cols)}")
print("Features:", feature_cols)


In [ ]:
# =============================================================================
# 3. OPTIMIZACIÓN DE HIPERPARÁMETROS CON OPTUNA
# =============================================================================

print("\n" + "="*70)
print("OPTIMIZACIÓN DE HIPERPARÁMETROS CON OPTUNA") 
print("="*70)

def safe_training_step(model, batch_x, batch_y, optimizer, criterion, device):
    """Paso de entrenamiento seguro con manejo de NaN"""
    
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)
    
    # Validar entrada
    if not (validate_tensor_data(batch_x, "batch_x") and validate_tensor_data(batch_y, "batch_y")):
        return float('inf')
    
    optimizer.zero_grad()
    
    try:
        outputs = model(batch_x)
        
        # Validar salidas
        if not validate_tensor_data(outputs, "outputs"):
            return float('inf')
        
        loss = criterion(outputs, batch_y)
        
        # Validar pérdida
        if torch.isnan(loss) or torch.isinf(loss):
            print("Loss contiene NaN o Inf, saltando batch")
            return float('inf')
        
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        # Verificar gradientes
        total_norm = 0
        for p in model.parameters():
            if p.grad is not None:
                param_norm = p.grad.data.norm(2)
                total_norm += param_norm.item() ** 2
        total_norm = total_norm ** (1. / 2)
        
        if np.isnan(total_norm) or np.isinf(total_norm):
            print("Gradientes contienen NaN o Inf, saltando optimización")
            optimizer.zero_grad()
            return float('inf')
        
        optimizer.step()
        return loss.item()
        
    except Exception as e:
        print(f"Error en paso de entrenamiento: {e}")
        optimizer.zero_grad()
        return float('inf')

def objective(trial):
    """Función objetivo para Optuna con manejo de errores mejorado"""
    
    # Hiperparámetros a optimizar
    seq_length = trial.suggest_int('seq_length', 8, 48)
    hidden_size = trial.suggest_int('hidden_size', 32, 96, step=16)
    num_layers = trial.suggest_int('num_layers', 1, 3)
    dropout = trial.suggest_float('dropout', 0.1, 0.4)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_int('batch_size', 16, 64, step=16)
    
    try:
        # Preparar datos
        train_dataset, val_dataset, scaler_features, scaler_target = prepare_data_for_model(
            df_train, df_test, feature_cols, target_col, seq_length)
        
        if len(train_dataset) == 0 or len(val_dataset) == 0:
            print("Dataset vacío, retornando penalización")
            return float('inf')
        
        # DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        
        # Modelo
        model = PeepholeLSTM(
            input_size=len(feature_cols),
            hidden_size=hidden_size,
            num_layers=num_layers,
            output_size=1,
            dropout=dropout
        ).to(device)
        
        # Optimizador y función de pérdida
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
        criterion = nn.MSELoss()
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.7)
        
        # Entrenamiento
        model.train()
        epochs = 50  # Reducido para optimización más rápida
        
        best_val_loss = float('inf')
        patience_count = 0
        max_patience = 25
        
        for epoch in range(epochs):
            total_loss = 0
            valid_batches = 0
            
            for batch_x, batch_y in train_loader:
                loss = safe_training_step(model, batch_x, batch_y, optimizer, criterion, device)
                if loss != float('inf'):
                    total_loss += loss
                    valid_batches += 1
            
            if valid_batches == 0:
                print("No hay batches válidos, terminando trial")
                return float('inf')
            
            avg_train_loss = total_loss / valid_batches
            
            # Validación
            model.eval()
            val_loss = 0
            valid_val_batches = 0
            
            with torch.no_grad():
                for batch_x, batch_y in val_loader:
                    batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                    
                    if validate_tensor_data(batch_x) and validate_tensor_data(batch_y):
                        outputs = model(batch_x)
                        if validate_tensor_data(outputs):
                            loss = criterion(outputs, batch_y)
                            if not (torch.isnan(loss) or torch.isinf(loss)):
                                val_loss += loss.item()
                                valid_val_batches += 1
            
            if valid_val_batches == 0:
                return float('inf')
            
            val_loss /= valid_val_batches
            scheduler.step(val_loss)
            model.train()
            
            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_count = 0
            else:
                patience_count += 1
                if patience_count >= max_patience:
                    break
            
            # Reporte intermedio para pruning
            if epoch % 10 == 0:
                trial.report(val_loss, epoch)
                if trial.should_prune():
                    raise optuna.exceptions.TrialPruned()
        
        return best_val_loss
        
    except optuna.exceptions.TrialPruned:
        raise
    except Exception as e:
        print(f"Error en trial: {e}")
        return float('inf')

# Ejecutar optimización
print("Iniciando optimización de hiperparámetros...")
study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner())

# Reducir número de trials para ejemplo
n_trials = 40
print(f"Ejecutando {n_trials} trials...")

study.optimize(objective, n_trials=n_trials)  # 1 hora máximo

# Resultados de la optimización
print("\n--- RESULTADOS DE LA OPTIMIZACIÓN ---")
print(f"Número de trials completados: {len(study.trials)}")
print(f"Mejor valor: {study.best_value:.6f}")
print("Mejores hiperparámetros:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

In [ ]:
# =============================================================================
# 4. ENTRENAMIENTO FINAL CON TODOS LOS DATOS
# =============================================================================

print("\n" + "="*70)
print("ENTRENAMIENTO FINAL CON TODOS LOS DATOS")
print("="*70)

# Usar mejores hiperparámetros
best_params = study.best_params
print("Usando mejores hiperparámetros encontrados...")

# Combinar todos los datos para entrenamiento final
all_data = df_train.copy() #pd.concat([df_train, df_test], ignore_index=True)
all_data = clean_and_impute_data(all_data)
print(f"Datos totales para entrenamiento: {all_data.shape}")

# Preparar datos finales
seq_length = best_params['seq_length']
scaler_features_final = StandardScaler()
scaler_target_final = StandardScaler()

# Normalizar todos los datos
all_data[feature_cols] = scaler_features_final.fit_transform(all_data[feature_cols])
all_data[[target_col]] = scaler_target_final.fit_transform(all_data[[target_col]])

# Dataset final
final_dataset = TimeSeriesDataset(all_data, target_col, feature_cols, seq_length)
final_loader = DataLoader(final_dataset, batch_size=best_params['batch_size'], shuffle=True)

# Modelo final
final_model = PeepholeLSTM(
    input_size=len(feature_cols),
    hidden_size=best_params['hidden_size'],
    num_layers=best_params['num_layers'],
    output_size=1,
    dropout=best_params['dropout']
).to(device)

# Optimizador y criterio
final_optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'], weight_decay=1e-5)
final_criterion = nn.MSELoss()
final_scheduler = optim.lr_scheduler.ReduceLROnPlateau(final_optimizer, 'min', patience=10, factor=0.7)

# Entrenamiento final
print("Iniciando entrenamiento final...")
final_epochs = 200
train_losses = []

final_model.train()
for epoch in range(final_epochs):
    total_loss = 0
    valid_batches = 0
    
    for batch_x, batch_y in final_loader:
        loss = safe_training_step(final_model, batch_x, batch_y, final_optimizer, final_criterion, device)
        if loss != float('inf'):
            total_loss += loss
            valid_batches += 1
    
    if valid_batches == 0:
        print(f"No hay batches válidos en época {epoch+1}")
        continue
    
    avg_loss = total_loss / valid_batches
    train_losses.append(avg_loss)
    final_scheduler.step(avg_loss)
    
    if (epoch + 1) % 20 == 0:
        print(f"Época {epoch+1}/{final_epochs}, Loss: {avg_loss:.6f}")

# Visualizar curva de entrenamiento
plt.figure(figsize=(10, 6))
plt.plot(train_losses, linewidth=2, color='blue')
plt.title('Curva de Entrenamiento - Modelo Final', fontsize=14, fontweight='bold')
plt.xlabel('Época')
plt.ylabel('Loss (MSE)')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
from datetime import datetime

# =============================================================================
# 5. GENERACIÓN DE PREDICCIONES PARA 2022
# =============================================================================

print("\n" + "="*70)
print("GENERACIÓN DE PREDICCIONES PARA 2022")
print("="*70)

def generate_predictions_2022(model, scaler_features, scaler_target, 
                             all_data, feature_cols, seq_length, sample_submission):
    """Generar predicciones para todas las semanas de 2022 con manejo robusto"""
    
    model.eval()
    predictions = []
    
    # Obtener IDs únicos del archivo de submisión
    submission_ids = sample_submission['id'].unique()
    
    print(f"Generando predicciones para {len(submission_ids)} registros...")
    
    for i, submission_id in enumerate(submission_ids):
        try:
            # Extraer información del ID
            parts = submission_id.split('_')
            id_bar = int(parts[0])
            anio = int(parts[1])
            semana = int(parts[2])
            
            # Obtener datos históricos del barrio
            barrio_data = all_data[all_data['id_bar'] == id_bar].sort_values('fecha')
            
            if len(barrio_data) >= seq_length:
                # Tomar las últimas seq_length observaciones
                recent_data = barrio_data.tail(seq_length)
                features = recent_data[feature_cols].values
                
                # Verificar datos válidos
                if not (np.isnan(features).any() or np.isinf(features).any()):
                    # Convertir a tensor
                    sequence = torch.FloatTensor(features).unsqueeze(0).to(device)
                    
                    # Predicción
                    with torch.no_grad():
                        pred = model(sequence)
                        
                        if validate_tensor_data(pred, f"prediction_{i}"):
                            pred_scaled = pred.cpu().numpy().flatten()[0]
                            
                            # Desnormalizar
                            try:
                                pred_original = scaler_target.inverse_transform([[pred_scaled]])[0][0]
                            except:
                                pred_original = 0.0
                            
                            # Asegurar que sea no negativo
                            pred_original = max(0, pred_original)
                            
                            predictions.append({
                                'id': submission_id,
                                'dengue': pred_original
                            })
                        else:
                            # Usar media del barrio si hay problemas con predicción
                            barrio_mean = barrio_data[target_col].mean() if len(barrio_data) > 0 else 0
                            pred_original = scaler_target.inverse_transform([[barrio_mean]])[0][0] if not np.isnan(barrio_mean) else 0
                            predictions.append({
                                'id': submission_id,
                                'dengue': max(0, pred_original)
                            })
                else:
                    # Si hay NaN en features, usar media del barrio
                    barrio_mean = barrio_data[target_col].mean() if len(barrio_data) > 0 else 0
                    pred_original = scaler_target.inverse_transform([[barrio_mean]])[0][0] if not np.isnan(barrio_mean) else 0
                    predictions.append({
                        'id': submission_id,
                        'dengue': max(0, pred_original)
                    })
            else:
                # Si no hay suficientes datos históricos, usar media general
                general_mean = all_data[target_col].mean()
                pred_original = scaler_target.inverse_transform([[general_mean]])[0][0] if not np.isnan(general_mean) else 0
                predictions.append({
                    'id': submission_id,
                    'dengue': max(0, pred_original)
                })
                
        except Exception as e:
            print(f"Error procesando {submission_id}: {e}")
            # Usar predicción por defecto
            predictions.append({
                'id': submission_id,
                'dengue': 0.1  # Valor mínimo positivo
            })
        
        # Progreso
        if (i + 1) % 100 == 0:
            print(f"Procesados {i + 1}/{len(submission_ids)} registros")
    
    return pd.DataFrame(predictions)

# Generar predicciones finales
final_predictions = generate_predictions_2022(
    final_model, scaler_features_final, scaler_target_final, 
    all_data, feature_cols, seq_length, sample_submission
)

print(f"Predicciones generadas: {len(final_predictions)}")
print("\nPrimeras predicciones:")
print(final_predictions.head(10))

print(f"\nEstadísticas de las predicciones:")
print(final_predictions['dengue'].describe())

# Verificar que no hay NaN en predicciones
nan_predictions = final_predictions['dengue'].isnull().sum()
if nan_predictions > 0:
    print(f"⚠ Encontrados {nan_predictions} valores NaN en predicciones, reemplazando con 0.1")
    final_predictions['dengue'] = final_predictions['dengue'].fillna(0.1)

# Verificar formato de submisión
print(f"\nVerificación del formato:")
print(f"Columnas requeridas: {list(sample_submission.columns)}")
print(f"Columnas generadas: {list(final_predictions.columns)}")
print(f"Filas requeridas: {len(sample_submission)}")
print(f"Filas generadas: {len(final_predictions)}")

# Visualizar distribución de predicciones
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(final_predictions['dengue'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
plt.title('Distribución de Predicciones de Dengue 2022')
plt.xlabel('Casos Predichos de Dengue')
plt.ylabel('Frecuencia')

plt.subplot(1, 2, 2)
plt.boxplot(final_predictions['dengue'])
plt.title('Boxplot de Predicciones de Dengue 2022')
plt.ylabel('Casos Predichos de Dengue')

plt.tight_layout()
plt.show()

# Guardar predicciones
output_file = f'predicciones_Peephole_LSTM_{datetime.now().strftime("%Y%m%d_%H%M")}.csv'
final_predictions.to_csv(output_file, index=False)
print(f"\n✓ Predicciones guardadas en: {output_file}")

In [ ]:
# =============================================================================
# 6. RESUMEN FINAL Y VALIDACIÓN
# =============================================================================

print("\n" + "="*70)
print("RESUMEN FINAL DEL MODELO")
print("="*70)

print("✓ ANÁLISIS EXPLORATORIO COMPLETADO")
print(f"  - Datos analizados: {df_train.shape[0]:,} observaciones de entrenamiento")
print(f"  - Período temporal: {df_train['anio'].min()}-{df_train['anio'].max()}")
print(f"  - Barrios únicos: {df_train['id_bar'].nunique()}")

print("\n✓ OPTIMIZACIÓN DE HIPERPARÁMETROS")
print(f"  - Trials ejecutados: {len(study.trials)}")
print(f"  - Mejor score de validación: {study.best_value:.6f}")
print("  - Mejores hiperparámetros:")
for key, value in best_params.items():
    print(f"    • {key}: {value}")

print("\n✓ MODELO FINAL ENTRENADO")
print(f"  - Arquitectura: Peephole LSTM con validación de NaN")
print(f"  - Características de entrada: {len(feature_cols)}")
print(f"  - Secuencias de entrenamiento: {len(final_dataset):,}")
print(f"  - Épocas de entrenamiento: {final_epochs}")
if train_losses:
    print(f"  - Loss final: {train_losses[-1]:.6f}")

print("\n✓ PREDICCIONES GENERADAS")
print(f"  - Predicciones para 2022: {len(final_predictions):,}")
print(f"  - Rango de predicciones: {final_predictions['dengue'].min():.2f} - {final_predictions['dengue'].max():.2f}")
print(f"  - Media de predicciones: {final_predictions['dengue'].mean():.2f}")
print(f"  - Valores NaN en predicciones: {final_predictions['dengue'].isnull().sum()}")
print(f"  - Archivo guardado: {output_file}")

print("\n✓ VALIDACIÓN DEL FORMATO")
formato_correcto = (
    len(final_predictions) == len(sample_submission) and
    list(final_predictions.columns) == list(sample_submission.columns) and
    final_predictions['dengue'].isnull().sum() == 0
)
print(f"  - Formato de submisión: {'✓ CORRECTO' if formato_correcto else '✗ INCORRECTO'}")

print(f"\n{'='*70}")
print("PROCESO COMPLETADO EXITOSAMENTE CON MANEJO ROBUSTO DE NaN")
print(f"{'='*70}")

# Información del modelo para uso futuro
model_info = {
    'best_params': best_params,
    'feature_cols': feature_cols,
    'seq_length': seq_length,
    'final_loss': train_losses[-1] if train_losses else 'N/A',
    'predictions_file': output_file,
    'nan_handling': 'Implemented'
}

print(f"\nInformación del modelo guardada para referencia futura:")
for key, value in model_info.items():
    print(f"  {key}: {value}")